In [1]:
import numpy as np
import scipy.io

In [2]:
data = scipy.io.loadmat('ex4data1.mat')

# x -> (5000, 400); y -> (5000,)
x, y = data['X'], data['y'].ravel()
M = len(y)

# X -> (401, 5000)
X = np.vstack((np.ones(M), x.T))

N = X.shape[0]
K = len(np.unique(y))

# Layers
L = 3

In [3]:
weights = scipy.io.loadmat('ex4weights.mat')

# theta_1 -> (25, 401); theta_2 -> (10, 26)
theta_1, theta_2 = weights['Theta1'], weights['Theta2']
thetas = [theta_1, theta_2]

In [165]:
theta_1[:,1:].shape, theta_1[:,1:].T.shape

((25, 400), (400, 25))

In [105]:

    
#Y[1000:1150,:]

In [223]:
np.array([0])[-2]

IndexError: index -2 is out of bounds for axis 0 with size 1

In [110]:
Y[:10, :]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int64)

# Feedforward and cost function

In [4]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))

In [5]:
lambda_ = 0

In [180]:
sum([1,2,3,4])

10

In [6]:
def nnCostFunction(X, y, thetas, M, N, L, K, lambda_):
    
    # Y -> (5000, 10)
    Y = np.zeros((M, K), dtype=np.int64)
    for i, row in enumerate(Y):
        row[y[i] - 1] = 1
    
    ## Fordward propagation
    activations = []
    a_i = X
    for i in range(L - 1):
        if i == (L - 2):
            # Last iteration, no bias
            a_i = sigmoid(thetas[i].dot(a_i))
        else:
            a_i = np.vstack((np.ones(M), sigmoid(thetas[i].dot(a_i))))

        activations.append(a_i)
            
    # Cost function
    h = a_i
    cost = .0
    for i in range(M):
        cost += -1. / M * (Y[i,:].dot(np.log(h[:,i])) + (1 - Y[i,:]).dot(np.log(1 - h[:,i])))
    
    ## Backward propagation
    deltas = []
    for i in reversed(range(L - 1)):
        if i == (L - 2):
            # output layer
            delta = h - y
        else:
            activation = activations[i][1:,:]
            theta = thetas[i - 1][:,1:]
            delta = ( theta.T.dot(deltas[i - 1]) ) * (activation * (1 - activation))
            
        deltas.append(delta)
        
    deltas.reverse()
    activations.insert(0,X)
    deltas.insert(0,np.array([0]))
    
    print('activation units:')
    for i, a in enumerate(activations):
        print(f'layer_{i + 1} : {a.shape}')
        
    print('deltas:')
    for i, d in enumerate(deltas):
        print(f'layer_{i + 1} : {d.shape}')
        
    capital_deltas = []
    grads = []
    for i in range(len(thetas)):
        theta = thetas[i][:,1:]
        capital_delta = deltas[i+1].dot(activations[i][1:,:].T)
        capital_deltas.append(capital_delta)
        
        grad = (1/M) * capital_delta
        if i > 0:
            grad += lambda_*theta
            
        grads.append(grad)
    
    print('capital deltas:')
    for i, cp in enumerate(capital_deltas):
        print(f'layer_{i + 1} : {cp.shape}')
        
    # Solucion mas corta que algun dia entenderé
    # cost = 1 / M * np.trace(- Y.dot(np.log(a_3)) - (1 - Y).dot(np.log(1 - a_3)))
        
    reg = lambda_ / (2 * M) * ( sum([(theta[:,1:]**2).sum() for theta in thetas]) )
    return cost + reg, grads

In [7]:
nnCostFunction(X, y, thetas, M, N, L, K, lambda_)

activation units:
layer_1 : (401, 5000)
layer_2 : (26, 5000)
layer_3 : (10, 5000)
deltas:
layer_1 : (1,)
layer_2 : (25, 5000)
layer_3 : (10, 5000)
capital deltas:
layer_1 : (25, 400)
layer_2 : (10, 25)


(0.28762916516131876,
 [array([[ 0.00000000e+00,  0.00000000e+00,  1.06528714e-07, ...,
          -8.87198712e-06, -2.30425737e-07,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  2.69263628e-08, ...,
           4.33506773e-07,  1.59620926e-08,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  8.50022670e-09, ...,
           8.49204752e-07,  1.23790777e-08,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  6.84422467e-08, ...,
          -4.69241164e-07, -4.06536573e-08,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  1.08049456e-07, ...,
          -7.76678427e-06, -6.55229503e-07,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  1.04201585e-07, ...,
          -8.01141602e-06, -1.12037580e-07,  0.00000000e+00]]),
  array([[-2.47995995, -1.46450736, -3.31558531, -3.36390088, -3.94132509,
          -3.29120936, -1.52470956, -0.98151833, -3.97675246, -2.41418641,
          -3.46601709, -3.91944608, -1.43464

## Regularized cost function

In [8]:
lambda_ = 1

In [9]:
nnCostFunction(X, y, thetas, M, N, L, K, lambda_)

activation units:
layer_1 : (401, 5000)
layer_2 : (26, 5000)
layer_3 : (10, 5000)
deltas:
layer_1 : (1,)
layer_2 : (25, 5000)
layer_3 : (10, 5000)
capital deltas:
layer_1 : (25, 400)
layer_2 : (10, 25)


(0.3837698590909235,
 [array([[ 0.00000000e+00,  0.00000000e+00,  1.06528714e-07, ...,
          -8.87198712e-06, -2.30425737e-07,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  2.69263628e-08, ...,
           4.33506773e-07,  1.59620926e-08,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  8.50022670e-09, ...,
           8.49204752e-07,  1.23790777e-08,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  6.84422467e-08, ...,
          -4.69241164e-07, -4.06536573e-08,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  1.08049456e-07, ...,
          -7.76678427e-06, -6.55229503e-07,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  1.04201585e-07, ...,
          -8.01141602e-06, -1.12037580e-07,  0.00000000e+00]]),
  array([[-3.69240493, -1.56637867, -5.68408617, -4.42168217, -6.14956139,
          -2.72737102, -0.31365662,  1.22879164, -3.53219089, -3.59663514,
          -2.42312596, -5.52503364, -0.130448

# Backpropagation

## Sigmoid gradient

In [190]:
sigmoidGradient = lambda x: sigmoid(x) * (1 - sigmoid(x))

In [192]:
sigmoidGradient(np.array([-1, -0.5, 0, 0.5, 1]))

array([0.19661193, 0.23500371, 0.25      , 0.23500371, 0.19661193])

## Random initialization

In [194]:
eps_init = .12

In [195]:
randomInitialization = lambda eps_init, layer_1_size, layer_2_size: np.random.rand(layer_2_size, layer_1_size) * (2 * eps_init) - eps_init

In [198]:
# (25, 400)
initial_theta_1 = randomInitialization(eps_init, N - 1, 25)
# (25, 10)
initial_theta_2 = randomInitialization(eps_init, 25, K)

((25, 400), (10, 25))

In [253]:
lambda_ = 3

In [254]:
nnCostFunction(X, y, thetas, M, N, L, K, lambda_)

activation units:
layer_1 : (401, 5000)
layer_2 : (26, 5000)
layer_3 : (10, 5000)
deltas:
layer_1 : (1,)
layer_2 : (25, 5000)
layer_3 : (10, 5000)
capital deltas:
layer_1 : (25, 400)
layer_2 : (10, 25)


(0.5760512469501329,
 [array([[ 0.00000000e+00,  0.00000000e+00,  1.06528714e-07, ...,
          -8.87198712e-06, -2.30425737e-07,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  2.69263628e-08, ...,
           4.33506773e-07,  1.59620926e-08,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  8.50022670e-09, ...,
           8.49204752e-07,  1.23790777e-08,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  6.84422467e-08, ...,
          -4.69241164e-07, -4.06536573e-08,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  1.08049456e-07, ...,
          -7.76678427e-06, -6.55229503e-07,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  1.04201585e-07, ...,
          -8.01141602e-06, -1.12037580e-07,  0.00000000e+00]]),
  array([[-6.11729490e+00, -1.77012128e+00, -1.04210879e+01,
          -6.53724475e+00, -1.05660340e+01, -1.59969434e+00,
           2.10844926e+00,  5.64941159e+00, -2.64306777e+00,
          -5

# Accuracy

In [ ]:
def accuracy(predictions, y):
    selected_y = np.argmax(predictions, axis=1) + 1
    return (selected_y == y).mean()

In [ ]:
accuracy(predictions.T, y)